# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2
l1_lambda_regularization = 0.10
def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc3'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.3)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.3
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(12751, 336)
(12751, 1)
Cross-Validation data (X, y)
(30, 336)
(30, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.59060403 -0.04454685 -0.33462867 ... -0.45454545  0.33333333
   0.39741329]
 [-0.36912752  0.0640041  -0.22495682 ... -0.27272727  0.25581395
   0.57142857]
 [-0.23489933 -0.13210445 -0.17918826 ... -0.09090909  0.11627907
   0.53057025]
 [-0.23489933 -0.3983615  -0.27979275 ... -0.27272727  0.31782946
   0.6255144 ]
 [-0.22147651 -0.3952893  -0.09196891 ... -0.81818182  0.34883721
   0.14609053]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Cross-Validation data (X, y)
[[-0.22147651 -0.24987199 -0.09585492 ... -0.45454545  0.20930233
   0.13786008]
 [-0.10738255 -0.34767025 -0.31131261 ... -0.63636364  0.2248062
   0.44121105]
 [-0.57718

# Train the model and test some Tunable Model functionalities

In [9]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        print("Logs:", logs.items())
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        print("Val_logs:", val_logs)
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


In [10]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = nFeatures*tModel.data_handler.sequence_length


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

now = datetime.now()
#tensorboard = TensorBoard(log_dir="logs/viv_log/")
#log_dir="./logs".format(time())
tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)

tModel.epochs = 200
tModel.train_model(learningRate_scheduler=lrate, verbose=1,tensorboard = tensorboard)




training with cv
Train on 12751 samples, validate on 30 samples
Epoch 1/200
12751/12751 [==============================] - 0s 28us/step - loss: 8554.4185 - mean_squared_error: 8505.9341 - val_loss: 11738.7490 - val_mean_squared_error: 11691.9268
Logs: dict_items([('loss', 8554.418463119928), ('mean_squared_error', 8505.934089881432), ('val_loss', 11738.7490234375), ('val_mean_squared_error', 11691.9267578125)])
Val_logs: {'mean_squared_error': 11691.9267578125, 'loss': 11738.7490234375}
Epoch 2/200
12751/12751 [==============================] - 0s 7us/step - loss: 7706.1041 - mean_squared_error: 7659.9622 - val_loss: 10419.8525 - val_mean_squared_error: 10373.3398
Logs: dict_items([('loss', 7706.1040557940305), ('mean_squared_error', 7659.962175339496), ('val_loss', 10419.8525390625), ('val_mean_squared_error', 10373.33984375)])
Val_logs: {'mean_squared_error': 10373.33984375, 'loss': 10419.8525390625}
Epoch 3/200
12751/12751 [==============================] - 0s 7us/step - loss: 6621.

12751/12751 [==============================] - 0s 9us/step - loss: 714.9349 - mean_squared_error: 621.4278 - val_loss: 2198.9221 - val_mean_squared_error: 2105.2075
Logs: dict_items([('loss', 714.9349465036979), ('mean_squared_error', 621.427833019224), ('val_loss', 2198.922119140625), ('val_mean_squared_error', 2105.20751953125)])
Val_logs: {'mean_squared_error': 2105.20751953125, 'loss': 2198.922119140625}
Epoch 21/200
12751/12751 [==============================] - 0s 9us/step - loss: 668.4751 - mean_squared_error: 574.5479 - val_loss: 2158.9824 - val_mean_squared_error: 2064.8357
Logs: dict_items([('loss', 668.475123260285), ('mean_squared_error', 574.5478600780316), ('val_loss', 2158.982421875), ('val_mean_squared_error', 2064.835693359375)])
Val_logs: {'mean_squared_error': 2064.835693359375, 'loss': 2158.982421875}
Epoch 22/200
12751/12751 [==============================] - 0s 10us/step - loss: 632.7316 - mean_squared_error: 538.4533 - val_loss: 2116.1682 - val_mean_squared_error

12751/12751 [==============================] - 0s 10us/step - loss: 519.0517 - mean_squared_error: 421.6759 - val_loss: 1938.4119 - val_mean_squared_error: 1840.9172
Logs: dict_items([('loss', 519.0516635744293), ('mean_squared_error', 421.6758608120618), ('val_loss', 1938.411865234375), ('val_mean_squared_error', 1840.917236328125)])
Val_logs: {'mean_squared_error': 1840.917236328125, 'loss': 1938.411865234375}
Epoch 40/200
12751/12751 [==============================] - 0s 9us/step - loss: 516.7819 - mean_squared_error: 419.2056 - val_loss: 1921.0166 - val_mean_squared_error: 1823.3273
Logs: dict_items([('loss', 516.7819387025903), ('mean_squared_error', 419.2056400495626), ('val_loss', 1921.0166015625), ('val_mean_squared_error', 1823.3272705078125)])
Val_logs: {'mean_squared_error': 1823.3272705078125, 'loss': 1921.0166015625}
Epoch 41/200
12751/12751 [==============================] - 0s 9us/step - loss: 514.4724 - mean_squared_error: 416.6252 - val_loss: 1924.5215 - val_mean_squar

12751/12751 [==============================] - 0s 10us/step - loss: 472.6147 - mean_squared_error: 366.4681 - val_loss: 1830.8372 - val_mean_squared_error: 1724.3257
Logs: dict_items([('loss', 472.6146973723683), ('mean_squared_error', 366.46807367819804), ('val_loss', 1830.837158203125), ('val_mean_squared_error', 1724.32568359375)])
Val_logs: {'mean_squared_error': 1724.32568359375, 'loss': 1830.837158203125}
Epoch 59/200
12751/12751 [==============================] - 0s 10us/step - loss: 469.5940 - mean_squared_error: 362.6644 - val_loss: 1827.3114 - val_mean_squared_error: 1719.9231
Logs: dict_items([('loss', 469.59404466659095), ('mean_squared_error', 362.66439907889804), ('val_loss', 1827.3114013671875), ('val_mean_squared_error', 1719.923095703125)])
Val_logs: {'mean_squared_error': 1719.923095703125, 'loss': 1827.3114013671875}
Epoch 60/200
12751/12751 [==============================] - 0s 10us/step - loss: 466.3917 - mean_squared_error: 358.5234 - val_loss: 1817.2443 - val_mea

12751/12751 [==============================] - 0s 6us/step - loss: 425.1908 - mean_squared_error: 308.2539 - val_loss: 1696.0457 - val_mean_squared_error: 1579.0781
Logs: dict_items([('loss', 425.190815794338), ('mean_squared_error', 308.2539195761607), ('val_loss', 1696.045654296875), ('val_mean_squared_error', 1579.078125)])
Val_logs: {'mean_squared_error': 1579.078125, 'loss': 1696.045654296875}
Epoch 78/200
12751/12751 [==============================] - 0s 6us/step - loss: 423.7776 - mean_squared_error: 306.7899 - val_loss: 1693.7620 - val_mean_squared_error: 1576.6998
Logs: dict_items([('loss', 423.7775605418394), ('mean_squared_error', 306.7899177532721), ('val_loss', 1693.761962890625), ('val_mean_squared_error', 1576.6998291015625)])
Val_logs: {'mean_squared_error': 1576.6998291015625, 'loss': 1693.761962890625}
Epoch 79/200
12751/12751 [==============================] - 0s 6us/step - loss: 422.1312 - mean_squared_error: 305.1210 - val_loss: 1674.7843 - val_mean_squared_error: 

...

12751/12751 [==============================] - 0s 9us/step - loss: 395.9776 - mean_squared_error: 283.8676 - val_loss: 1625.7474 - val_mean_squared_error: 1513.6512
Logs: dict_items([('loss', 395.9775786652265), ('mean_squared_error', 283.8675910974239), ('val_loss', 1625.7474365234375), ('val_mean_squared_error', 1513.6512451171875)])
Val_logs: {'mean_squared_error': 1513.6512451171875, 'loss': 1625.7474365234375}
Epoch 116/200
12751/12751 [==============================] - 0s 7us/step - loss: 395.8814 - mean_squared_error: 283.8053 - val_loss: 1625.9355 - val_mean_squared_error: 1513.8853
Logs: dict_items([('loss', 395.881378365296), ('mean_squared_error', 283.8052511118728), ('val_loss', 1625.935546875), ('val_mean_squared_error', 1513.88525390625)])
Val_logs: {'mean_squared_error': 1513.88525390625, 'loss': 1625.935546875}
Epoch 117/200
12751/12751 [==============================] - 0s 8us/step - loss: 395.7725 - mean_squared_error: 283.7322 - val_loss: 1626.0591 - val_mean_squared

12751/12751 [==============================] - 0s 9us/step - loss: 393.9975 - mean_squared_error: 282.7235 - val_loss: 1619.8739 - val_mean_squared_error: 1508.6238
Logs: dict_items([('loss', 393.9975244047763), ('mean_squared_error', 282.72352587293244), ('val_loss', 1619.8739013671875), ('val_mean_squared_error', 1508.623779296875)])
Val_logs: {'mean_squared_error': 1508.623779296875, 'loss': 1619.8739013671875}
Epoch 135/200
12751/12751 [==============================] - 0s 7us/step - loss: 393.9023 - mean_squared_error: 282.6852 - val_loss: 1621.1522 - val_mean_squared_error: 1509.9640
Logs: dict_items([('loss', 393.90231065239294), ('mean_squared_error', 282.6851993202088), ('val_loss', 1621.1522216796875), ('val_mean_squared_error', 1509.9639892578125)])
Val_logs: {'mean_squared_error': 1509.9639892578125, 'loss': 1621.1522216796875}
Epoch 136/200
12751/12751 [==============================] - 0s 9us/step - loss: 393.7714 - mean_squared_error: 282.6100 - val_loss: 1621.4181 - val

12751/12751 [==============================] - 0s 9us/step - loss: 391.8080 - mean_squared_error: 281.5547 - val_loss: 1619.7803 - val_mean_squared_error: 1509.5594
Logs: dict_items([('loss', 391.80796148733816), ('mean_squared_error', 281.5547384112949), ('val_loss', 1619.7802734375), ('val_mean_squared_error', 1509.5594482421875)])
Val_logs: {'mean_squared_error': 1509.5594482421875, 'loss': 1619.7802734375}
Epoch 154/200
12751/12751 [==============================] - 0s 9us/step - loss: 391.7004 - mean_squared_error: 281.5078 - val_loss: 1616.4401 - val_mean_squared_error: 1506.2734
Logs: dict_items([('loss', 391.7003677412441), ('mean_squared_error', 281.50776625812665), ('val_loss', 1616.4400634765625), ('val_mean_squared_error', 1506.2734375)])
Val_logs: {'mean_squared_error': 1506.2734375, 'loss': 1616.4400634765625}
Epoch 155/200
12751/12751 [==============================] - 0s 9us/step - loss: 391.5751 - mean_squared_error: 281.4299 - val_loss: 1617.0070 - val_mean_squared_er

12751/12751 [==============================] - 0s 10us/step - loss: 389.4405 - mean_squared_error: 280.3498 - val_loss: 1613.8021 - val_mean_squared_error: 1504.7469
Logs: dict_items([('loss', 389.44051258349526), ('mean_squared_error', 280.3497981431297), ('val_loss', 1613.8021240234375), ('val_mean_squared_error', 1504.7469482421875)])
Val_logs: {'mean_squared_error': 1504.7469482421875, 'loss': 1613.8021240234375}
Epoch 173/200
12751/12751 [==============================] - 0s 7us/step - loss: 389.3014 - mean_squared_error: 280.2678 - val_loss: 1613.9596 - val_mean_squared_error: 1504.9700
Logs: dict_items([('loss', 389.301363338518), ('mean_squared_error', 280.267786650216), ('val_loss', 1613.9595947265625), ('val_mean_squared_error', 1504.969970703125)])
Val_logs: {'mean_squared_error': 1504.969970703125, 'loss': 1613.9595947265625}
Epoch 174/200
12751/12751 [==============================] - 0s 15us/step - loss: 389.1809 - mean_squared_error: 280.2134 - val_loss: 1612.7015 - val_

12751/12751 [==============================] - 0s 10us/step - loss: 386.8889 - mean_squared_error: 279.0464 - val_loss: 1609.1826 - val_mean_squared_error: 1501.4003
Logs: dict_items([('loss', 386.88888213778654), ('mean_squared_error', 279.0464336451208), ('val_loss', 1609.1826171875), ('val_mean_squared_error', 1501.4002685546875)])
Val_logs: {'mean_squared_error': 1501.4002685546875, 'loss': 1609.1826171875}
Epoch 192/200
12751/12751 [==============================] - 0s 9us/step - loss: 386.7604 - mean_squared_error: 278.9946 - val_loss: 1610.5426 - val_mean_squared_error: 1502.8323
Logs: dict_items([('loss', 386.7604114364039), ('mean_squared_error', 278.99461193980727), ('val_loss', 1610.5426025390625), ('val_mean_squared_error', 1502.832275390625)])
Val_logs: {'mean_squared_error': 1502.832275390625, 'loss': 1610.5426025390625}
Epoch 193/200
12751/12751 [==============================] - 0s 9us/step - loss: 386.6186 - mean_squared_error: 278.9542 - val_loss: 1606.0702 - val_mean

In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

30/30 [==============================] - 0s 0us/step
scores
Engine 1, Predicted RUL 98.21092987060547, Rounded RUL 98.0, Real RUL 112.0
Engine 2, Predicted RUL 127.51885223388672, Rounded RUL 127.0, Real RUL 98.0
Engine 3, Predicted RUL 107.36609649658203, Rounded RUL 107.0, Real RUL 69.0
Engine 4, Predicted RUL 113.54732513427734, Rounded RUL 113.0, Real RUL 82.0
Engine 5, Predicted RUL 14.380489349365234, Rounded RUL 14.0, Real RUL 91.0
Engine 6, Predicted RUL 78.87421417236328, Rounded RUL 78.0, Real RUL 93.0
Engine 7, Predicted RUL 133.71713256835938, Rounded RUL 133.0, Real RUL 91.0
Engine 8, Predicted RUL 27.983259201049805, Rounded RUL 27.0, Real RUL 95.0
Engine 9, Predicted RUL 94.62877655029297, Rounded RUL 94.0, Real RUL 111.0
Engine 10, Predicted RUL 127.55299377441406, Rounded RUL 127.0, Real RUL 96.0
Engine 11, Predicted RUL 28.584745407104492, Rounded RUL 28.0, Real RUL 97.0
Engine 12, Predicted RUL 111.80402374267578, Rounded RUL 111.0, Real RUL 124.0
Engine 13, Predicte

In [12]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 156us/step
scores
Engine 1, Predicted RUL 117.5299301147461, Rounded RUL 117.0, Real RUL 112.0
Engine 2, Predicted RUL 118.06590270996094, Rounded RUL 118.0, Real RUL 98.0
Engine 3, Predicted RUL 60.859100341796875, Rounded RUL 60.0, Real RUL 69.0
Engine 4, Predicted RUL 84.0525131225586, Rounded RUL 84.0, Real RUL 82.0
Engine 5, Predicted RUL 90.41642761230469, Rounded RUL 90.0, Real RUL 91.0
Engine 6, Predicted RUL 102.31657409667969, Rounded RUL 102.0, Real RUL 93.0
Engine 7, Predicted RUL 111.49030303955078, Rounded RUL 111.0, Real RUL 91.0
Engine 8, Predicted RUL 87.96669006347656, Rounded RUL 87.0, Real RUL 95.0
Engine 9, Predicted RUL 106.79798889160156, Rounded RUL 106.0, Real RUL 111.0
Engine 10, Predicted RUL 95.19841766357422, Rounded RUL 95.0, Real RUL 96.0
Engine 11, Predicted RUL 87.22893524169922, Rounded RUL 87.0, Real RUL 97.0
Engine 12, Predicted RUL 103.01997375488281, Rounded RUL 103.0, Real RUL 124.0
Engine 13, Predicte